In [181]:
import datetime as dt

# data analysis imports
import pymysql as msql
import pandas as pd
import numpy as np

# ml imports
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer

import pdb

% matplotlib inline

In [182]:
"""
Tables
------
cb_acquisitions
cb_degrees
cb_funding_rounds
cb_funds
cb_ipos
cb_milestones
cb_objects
cb_offices
cb_people
cb_relationships
cb_investments
"""

'\nTables\n------\ncb_acquisitions\ncb_degrees\ncb_funding_rounds\ncb_funds\ncb_ipos\ncb_milestones\ncb_objects\ncb_offices\ncb_people\ncb_relationships\ncb_investments\n'

### Helper Methods

In [183]:
# Create a sql.txt file
# Put the password to root@localhost on the first line
# Put the name of your Crunchbase db in the second line

with open('sql.txt','r') as f:
    pwd = f.readline()
    pwd = pwd.rstrip()
    db_name = f.readline()
    db_name = db_name.rstrip()

In [184]:
def establish_connection():
    """
        Helper to establish connection with local mysql database. Returns cursor object. Publicly available
        dataset so password commits are ignored.
    """
    conn = msql.connect(host="localhost", user="root", password=pwd, db=db_name, 
                        cursorclass=msql.cursors.DictCursor)
    return conn.cursor()

In [185]:
def fetch_dataframe(cursor):
    """
        Helper to fetch from database and return pandas dataframe.
        
        Parameters
        ----------
        cursor : {pymysql.connection.cursor}
            primary cursor obj
    """
    return pd.DataFrame(cursor.fetchall())

### Data Ingestion
Create dataframes for all of the db tables.

In [186]:
crs = establish_connection()

In [187]:
crs.execute("select * from cb_objects")
df_objects = fetch_dataframe(crs)

In [188]:
crs.execute("select * from cb_investments")
df_investments = fetch_dataframe(crs)

In [189]:
crs.execute("select * from cb_relationships")
df_relationships = fetch_dataframe(crs)

In [209]:
crs.execute("select * from cb_acquisitions")
df_acquisitions = fetch_dataframe(crs)

In [229]:
crs.execute("select * from cb_degrees")
df_degrees = fetch_dataframe(crs)

In [192]:
crs.execute("select * from cb_people")
df_people = fetch_dataframe(crs)

In [243]:
crs.execute("select * from cb_funds")
df_funds = fetch_dataframe(crs)

In [252]:
crs.execute("select * from cb_funding_rounds")
df_funrnds = fetch_dataframe(crs)

crs.execute("select * from cb_ipos")
df_ipos = fetch_dataframe(crs)

crs.execute("select * from cb_milestones")
df_milestones = fetch_dataframe(crs)

crs.execute("select * from cb_offices")
df_offices = fetch_dataframe(crs)

### Breaking up Objects table

In [210]:
df_all_companies = df_objects.loc[df_objects['entity_type']=='Company']
df_closed_companies = df_all_companies.loc[df_all_companies['status']=='closed']

df_acquired_companies = df_all_companies.loc[df_all_companies['status']=='acquired']
df_ipo_companies = df_all_companies.loc[df_all_companies['status']=='ipo']

df_all_fin_orgs = df_objects.loc[df_objects['entity_type']=='FinancialOrg']
df_all_people = df_objects.loc[df_objects['entity_type']=='Person']

# don't use df_objects anymore, use one of the above

### Checking for table consistency

In [211]:
# Check that df_acquired_companies matches up perfectly with df_acquisitions

df_acquired_tings = df_all_companies.loc[df_all_companies['id'].isin(df_acquisitions.acquired_object_id)]

# should be empty set
print np.setdiff1d(df_acquired_tings.id, df_acquired_companies.id)

# should also be empty but isn't for some reason..
# probably because those rows were deleted in Objects but not in Acquisitions
print np.setdiff1d(df_acquisitions.acquired_object_id, df_acquired_tings.id)
deleted_companies = np.setdiff1d(df_acquisitions.acquired_object_id, df_acquired_tings.id)

# should be all False
print np.in1d(deleted_companies, df_acquired_companies.id)

# so we can go ahead and delete these from df_acquisitions
acq_companies = np.setdiff1d(df_acquisitions.acquired_object_id, deleted_companies)
df_acquisitions = df_acquisitions[df_acquisitions.acquired_object_id.isin(acq_companies)]

# should be empty now--so we good
print np.setdiff1d(df_acquisitions.acquired_object_id, df_acquired_tings.id)

[]
['' 'c:11893' 'c:1429' 'c:16260' 'c:20393' 'c:21988' 'c:246942' 'c:247635'
 'c:25378' 'c:2619' 'c:29305' 'c:3822' 'c:3930' 'c:40599' 'c:48584'
 'c:66959' 'c:74735' 'c:74780']
[False False False False False False False False False False False False
 False False False False False False]
[]


In [196]:
# same checks for ipo companies
df_ipo_tings = df_all_companies.loc[df_all_companies['id'].isin(df_ipos.object_id.unique())]

# should be empty but is not
print np.setdiff1d(df_ipo_tings.id, df_ipo_companies.id)
# df_ipo_tings is bigger
print df_ipo_tings.shape, df_ipo_companies.shape
# this is because some companies that have been acquired have gone public
# under their own name (not their acquirers name)
# so we should prbly just take df_ipo_tings as our list of ipo_companies
df_ipo_companies = df_ipo_tings

['c:10241' 'c:10795' 'c:10855' 'c:12178' 'c:13219' 'c:13389' 'c:13402'
 'c:13529' 'c:139713' 'c:143111' 'c:148188' 'c:14963' 'c:15436' 'c:16178'
 'c:1645' 'c:1705' 'c:17205' 'c:17274' 'c:17482' 'c:1790' 'c:18127'
 'c:19066' 'c:1942' 'c:200534' 'c:210969' 'c:212510' 'c:220208' 'c:223425'
 'c:225697' 'c:231215' 'c:23241' 'c:23756' 'c:23911' 'c:24879' 'c:25106'
 'c:254996' 'c:25744' 'c:25821' 'c:261265' 'c:261911' 'c:2644' 'c:265625'
 'c:26754' 'c:26905' 'c:273194' 'c:27380' 'c:274037' 'c:274053' 'c:277914'
 'c:285065' 'c:29208' 'c:30923' 'c:32447' 'c:32496' 'c:32778' 'c:32951'
 'c:33792' 'c:34063' 'c:3484' 'c:35602' 'c:35663' 'c:36665' 'c:36666'
 'c:36920' 'c:37684' 'c:37796' 'c:37881' 'c:3827' 'c:39953' 'c:40358'
 'c:41' 'c:41391' 'c:41396' 'c:4498' 'c:47592' 'c:4823' 'c:4824' 'c:4850'
 'c:4999' 'c:51097' 'c:51364' 'c:528' 'c:5346' 'c:54506' 'c:55994' 'c:5617'
 'c:581' 'c:5990' 'c:61919' 'c:62372' 'c:63884' 'c:64486' 'c:67112'
 'c:68182' 'c:68190' 'c:68222' 'c:68733' 'c:71598' 'c:75577'

### Trimming all tables to only relevant rows

In [197]:
# compiling a list of all relevant companies
frames = [df_closed_companies, df_acquired_companies, df_ipo_companies]
df_relevant_companies = pd.concat(frames)
relevant_company_ids = df_relevant_companies.id.values

In [253]:
df_relevant_funrnds = df_funrnds[df_funrnds.object_id.isin(relevant_company_ids)]
df_relevant_investments = df_investments[df_investments.funded_object_id.isin(relevant_company_ids)]
df_relevant_milestones = df_milestones[df_milestones.object_id.isin(relevant_company_ids)]
df_relevant_offices = df_offices[df_offices.object_id.isin(relevant_company_ids)]

In [244]:
relevant_fund_ids = df_relevant_investments.investor_object_id.values
df_relevant_fin_orgs = df_fin_orgs[df_fin_orgs.id.isin(relevant_fund_ids)]
df_relevant_funds = df_funds[df_funds.object_id.isin(relevant_fund_ids)]

In [200]:
print set(df_relevant_companies.country_code.tolist())

set(['HUN', 'EGY', 'BEN', 'NGA', 'USA', 'DOM', 'LUX', 'ISR', 'GHA', 'SWE', 'DEU', 'PER', 'IDN', 'CRI', 'BGR', 'MEX', 'GBR', 'CAN', 'CYM', 'ISL', 'KOR', 'UMI', 'JOR', 'PAK', 'BEL', 'SGP', 'UGA', 'MDA', 'SOM', 'URY', 'PRT', 'POL', 'NLD', 'BMU', 'LBN', 'AIA', 'PHL', 'HKG', 'SAU', 'ARE', 'SVN', 'FRA', 'EST', 'LTU', 'CHE', 'AUT', 'ESP', 'TWN', 'MMR', 'HRV', 'CHL', 'DNK', 'ROM', 'AUS', 'LVA', 'IRL', 'CHN', 'CSS', 'VNM', 'GIN', 'FIN', 'VGB', 'THA', 'NZL', 'SYC', 'KEN', 'MAR', None, 'ZAF', 'NIC', 'TUR', 'JPN', 'ITA', 'ANT', 'NCL', 'NOR', 'TUN', 'ARG', 'IND', 'RUS', 'CZE', 'MYS', 'BRA', 'GRC'])


In [230]:
relevant_affiliation_ids = np.concatenate([relevant_company_ids, relevant_fund_ids])
df_relevant_relationships = df_relationships[df_relationships.relationship_object_id.isin(relevant_affiliation_ids)]
relevant_people_ids = df_relevant_relationships.person_object_id.values
df_relevant_people = df_people[df_people.object_id.isin(relevant_people_ids)]
df_relevant_degrees = df_degrees[df_degrees.object_id.isin(relevant_people_ids)]

In [206]:
# Only use the following from here onward:
# df_acquisitions
# df_ipos
# df_relevant_companies
# df_relevant_fin_orgs
# df_relevant_people
# df_relevant_funrnds
# df_relevant_investments
# df_relevant_milestones
# df_relevant_offices
# df relevant_funds
# df_relevant_relationships
# df_relevant_degrees

### Transformations

In [207]:
def acquisition_transform(df):
    """
        Method to transform acquisition dataframe for compatibility with remaining data set.
        
        Parameters
        ----------
        df : {pandas.DataFrame}
            acquisitons dataframe
    """
    
    # we trust master source (crunchbase) and we're not scraping source data
    # uninterested in creation/update dates
    drop_cols = ["source_url", "source_description", "created_at", 
                 "updated_at", "term_code", "price_currency_code", "acquisition_id"]
    for col in drop_cols:
        df.drop(col, inplace=True, axis=1)
    
    # convert all decimal values to float for sklearn compatibility
    decimal_cols = ["price_amount"]
    for col in decimal_cols:
        df[col] = df[col].apply(lambda x: float(x) if x is not None else x)

In [212]:
# drop all the non-USD ones to not have to deal with currency conversion
df_acquisitions=df_acquisitions[df_acquisitions.price_currency_code=="USD"]

acquisition_transform(df_acquisitions)

In [213]:
print df_acquisitions.shape
print df_acquisitions.columns

(9390, 5)
Index([u'acquired_at', u'acquired_object_id', u'acquiring_object_id', u'id',
       u'price_amount'],
      dtype='object')


In [226]:
df_relevant_degrees.shape

(58662, 6)

In [216]:
def degrees_transform(df, subject_ohe=False):
    """
        Method to transform degrees dataframe. All transformations are conducted on original dataframe.
        
        Parameters
        ----------
        df : {pandas.DataFrame}
            degrees dataframe
        
        subject_ohe : {bool}
            one hot encode subjects. default to False as degree subject might not necessarily be
            strong indicator of people success.
    """
    # uninterested in creation and update dates
    drop_cols = ["created_at", "updated_at"]
    for col in drop_cols:
        df.drop(col, inplace=True, axis=1)
    
    # significant date ranges for features
    df_relevant_degrees["subject"] = \
            df_relevant_degrees["subject"].apply(lambda x: [sub.strip() for sub in x.split(",")] if x is not None else [])    
    
    if subject_ohe:
        # binary features for subjects (multiple subjects are grouped together)
        subject_store = set(sum(df_degrees.subject.tolist(), []))

        # OHE for each subject
        for subj in subject_store:
            df["is_%s" % subj] = df["subject"].apply(lambda x: 1 if subj in x else 0)

        df.drop("subject", inplace=True, axis=1)

In [231]:
degrees_transform(df_relevant_degrees)

/Users/Krishn/py_env/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Krishn/py_env/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [245]:
def funds_transform(df):
    drop_cols = ["created_at", "updated_at", "source_description", "source_url", 
                 "name","raised_currency_code"]
    
    for col in drop_cols:
        df.drop(col, inplace=True, axis=1)
    
    decimal_cols = ["raised_amount"]
    for col in decimal_cols:
        df[col] = df[col].apply(lambda x: float(x) if x is not None else x)

In [247]:
df_relevant_funds = df_relevant_funds[df_relevant_funds.raised_currency_code == "USD"]
funds_transform(df_relevant_funds)

In [248]:
df_relevant_funds.head(2)

,fund_id,funded_at,id,object_id,raised_amount
0,1,2008-12-16,1,f:371,300000000.0
1,4,2008-12-17,4,f:17,200750000.0


In [249]:
df_relevant_funrnds.columns

Index([u'created_at', u'created_by', u'funded_at', u'funding_round_code',
       u'funding_round_id', u'funding_round_type', u'id', u'is_first_round',
       u'is_last_round', u'object_id', u'participants',
       u'post_money_currency_code', u'post_money_valuation',
       u'post_money_valuation_usd', u'pre_money_currency_code',
       u'pre_money_valuation', u'pre_money_valuation_usd', u'raised_amount',
       u'raised_amount_usd', u'raised_currency_code', u'source_description',
       u'source_url', u'updated_at'],
      dtype='object')

In [254]:
def funrnds_transform(df):
    drop_cols = ["created_at", "updated_at", "funding_round_type", "raised_amount",
                "raised_currency_code", "pre_money_currency_code", "post_money_currency_code",
                "source_url", "source_description", "created_by"]
    
    for col in drop_cols:
        df.drop(col, inplace=True, axis=1)

In [255]:
funrnds_transform(df_relevant_funrnds)

/Users/Krishn/py_env/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
